<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_6"><div id="image_img" class="header_image_6"></div></td>
        <td class="header_text"> EMG Analysis - Time and Frequency Parameters </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">extract&#9729;emg&#9729;muscular-activations</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

Muscles perform an essential role on movement, postural control and in the vital cardiorespiratory functions.
These three examples have something in common, which is, the origin of the muscular contraction is in the nervous impulse that reaches the muscle, triggering a sequence of physiological mechanisms that ultimately cause the muscle contraction.

In the last example (muscle in vital cardiorespiratory processes), myocardium (cardiac muscle) functions in an involuntary way, under the coordination of the autonomic nervous system.

The previously mentioned nervous impulse and the respective changes in electric potential can be monitored by ECG acquisition and analysis. 

However, for movement and postural control, another type of muscle goes into action, the skeletal muscle. In contrast to cardiac muscle, the action of skeletal muscles is voluntary, causing drastic differences in EMG signal when comparing to ECG, namely the inexistence of natural periodicity.

In this **<span class="color5">Jupyter Notebook</span>** it will be explained how some parameters can be extracted from EMG, both from time and frequency domain.

**List of EMG analysis parameters:**
+ Number of Muscular Activations;
+ Maximum, Minimum and Average duration of muscular activations;
+ Minimum, Maximum, Average and Standard Deviation values of EMG samples;
+ Root Mean Square (RMS) and Area under curve;
+ Total Power, Maximum Frequency and Median Frequency;

<hr>

<p class="steps">1 - Importation of the needed packages</p>

In [1]:
# biosignalsnotebooks python package
import biosignalsnotebooks as bsnb

# Scientific packages
from numpy import linspace, max, min, average, std, sum, sqrt, where, argmax
from scipy.integrate import cumtrapz
from scipy.signal import welch

<p class="steps">2 - Load of acquired EMG data</p>

In [2]:
# Load of data
data, header = bsnb.load_signal("emg_bursts", get_header=True)

<p class="steps">3 - Identification of mac address of the device and the channel used during acquisition</p>

In [3]:
channel = list(data.keys())[0]
device = header["device"]
resolution = int(header["resolution"][0])

In [4]:
from sty import fg, rs
print (fg(98,195,238) + "\033[1mDevice: \033[0m" + fg.rs + device + fg(98,195,238) + "\033[1m Channel: \033[0m" + fg.rs + str(channel) + fg(98,195,238) + "\033[1m Resolution: \033[0m" + fg.rs + str(resolution) + " bits")

Device: channeller Channel: CH3 Resolution: 16 bits


<p class="steps">4 - Storage of sampling frequency and acquired data inside variables</p>

In [5]:
# Sampling frequency and acquired data
fs = header["sampling rate"]

# Signal Samples
signal = bsnb.raw_to_phy("EMG", device, data[channel], resolution, option="mV") # Conversion to mV
time = linspace(0, len(signal) / fs, len(signal))

<p class="steps">5 -EMG parameter extraction
<br>5.1 -Detection and accounting of muscular activations</p>

In [6]:
burst_begin, burst_end = bsnb.detect_emg_activations(signal, fs, smooth_level=20, threshold_level=10, 
                                                     time_units=True, plot_result=True)[:2]

In [7]:
# Number of activation periods
print (fg(98,195,238) + "\033[1mNumber of Muscular Activations: \033[0m" + fg.rs + str(len(burst_begin)))

Number of Muscular Activations: 9


<p class="steps">5.2 -Maximum, Minimum and Average duration of muscular activation periods</p>

In [8]:
# Bursts Duration
bursts_time = burst_end - burst_begin

# Parameter extraction
max_time = max(bursts_time)
min_time = min(bursts_time)
avg_time = average(bursts_time)
std_time = std(bursts_time)

In [9]:
print (fg(98,195,238) + "\033[1m[Maximum, Minimum, Average] duration of Muscular Activations \033[0m" + fg.rs + " = [" + str(max_time) + ", " + str(min_time) + ", " + str(avg_time) + "] s")
print (fg(98,195,238) + "\033[1mStandard Deviation \033[0m" + fg.rs + "= " + str(std_time) + " s")

[Maximum, Minimum, Average] duration of Muscular Activations  = [1.3460471982607487, 0.6010210744091449, 0.9090318746055124] s
Standard Deviation = 0.22846855960280285 s


In [10]:
bsnb.plot_emg_graphical_durations(max_time, min_time, avg_time, std_time)

<p class="steps">5.3 - Maximum, Minimum, Average and Standard Deviation of EMG sample values</p>

In [11]:
# Maximum
max_sample_value = max(signal)

# Minimum
min_sample_value = min(signal)

# Average and Standard Deviation
avg_sample_value = average(signal)
std_sample_value = std(signal)

time_param_dict = {"Maximum EMG": max_sample_value, "Minimum EMG": min_sample_value, 
                   "Average EMG": avg_sample_value, "Standard Deviation EMG": std_sample_value}

In [12]:
print (fg(98,195,238) + "\033[1m[Maximum, Minimum, Average, Standard Deviation] mV \033[0m" + fg.rs + " = [" + str(max_sample_value) + ", " + str(min_sample_value) + ", " + str(avg_sample_value) + ", " + str(std_sample_value) + "] mV")

[Maximum, Minimum, Average, Standard Deviation] mV  = [0.478729248046875, -0.910400390625, 0.0016734834397682248, 0.06294409866652229] mV


In [13]:
bsnb.plot_emg_graphical_statistical(time, signal, max_sample_value, min_sample_value, avg_sample_value, std_sample_value)

<p class="steps">5.4 - Root Mean Square and Area under the curve (Signal Intensity Estimators)</p>

In [14]:
# Root Mean Square
rms = sqrt(sum(signal * signal) / len(signal))

# Area under the curve
area = cumtrapz(signal)

In [15]:
print (fg(98,195,238) + "\033[1mRoot Mean Square \033[0m" + fg.rs + " = " + str(rms) + " mV")
print (fg(98,195,238) + "\033[1mArea \033[0m" + fg.rs + " = " + str(area[-1]) + " mV.s")

Root Mean Square  = 0.06296634103839979 mV
Area  = 47.72003173828125 mV.s


In [16]:
bsnb.plot_emg_rms_area(time, signal, rms, area)

<p class="steps">5.5 - Total power and some reference points on the frequency domain</p>

In [17]:
# Signal Power Spectrum
f, P = welch(signal, fs=fs, window='hanning', noverlap=0, nfft=int(256.))

# Total Power and Median Frequency (Frequency that divides the spectrum into two regions with equal power)
area_freq = cumtrapz(P, f, initial=0)
total_power = area_freq[-1]
median_freq = f[where(area_freq >= total_power / 2)[0][0]]
f_max = f[argmax(P)]

In [18]:
print (fg(98,195,238) + "\033[1mTotal Power \033[0m" + fg.rs + " = " + str(total_power))
print (fg(98,195,238) + "\033[1m[Median Frequency, Maximum Power Frequency] \033[0m" + fg.rs + " = [" + str(median_freq) + ", " + str(f_max) + "] Hz")

Total Power  = 0.0038750259945056083
[Median Frequency, Maximum Power Frequency]  = [82.03125, 70.3125] Hz


In [19]:
bsnb.plot_emg_spect_freq(f, P, f_max, median_freq)

*This procedure can be automatically done by **emg_parameters** function in **extract** module of **<span class="color2">biosignalsnotebooks</span>** package*

In [20]:
bsnb.emg_parameters(signal, fs, raw_to_mv=False)

{'Number of Muscular Activations': 9,
 'Maximum Muscular Activation Duration': 1.3460471982607487,
 'Minimum Muscular Activation Duration': 0.6010210744091449,
 'Average Muscular Activation Duration': 0.9090318746055124,
 'Standard Deviation of Muscular Activation Duration': 0.22846855960280285,
 'Maximum Sample Value': 0.478729248046875,
 'Minimum Sample Value': -0.910400390625,
 'Average Sample Value': 0.0016734834397682248,
 'Standard Deviation Sample Value': 0.06294409866652229,
 'RMS': 0.06296634103839979,
 'Area': 47.72003173828125,
 'Total Power Spect': 0.0038750259945056083,
 'Median Frequency': 82.03125,
 'Maximum Power Frequency': 70.3125}

This set of parameters reveals interesting information about EMG signal, however you can extract much more features during your signal processing journey !

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> ! 

<span class="color6">**Auxiliary Code Segment (should not be replicated by the user)**</span>

In [21]:
from biosignalsnotebooks.__notebook_support__ import css_style_apply
css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................


In [22]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'],
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>